In [2]:
import gzip
import struct
import time

import numpy as np
import tensorflow as tf
import numpy.linalg as lng

from tensorflow import keras
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

from tensorflow.keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

print(tf.__version__)

1.11.0


In [3]:
dataset_cifar = np.load('cifar_dataset.npz')
Xtr = dataset_cifar['Xtr']    #training data
Str = dataset_cifar['Str']    #training label
Xts = dataset_cifar['Xts']    #test data
Yts = dataset_cifar['Yts']    #test label
print(Xtr.shape)
print(Str.shape)

(10000, 3072)
(10000, 1)


In [4]:
Xtr_cnn = Xtr.reshape(len(Xtr), 32, 32, 3).astype(np.float32) / 255.0
Xts_cnn = Xts.reshape(len(Xts), 32, 32, 3).astype(np.float32) / 255.0

In [5]:
Str_cnn = keras.utils.to_categorical(Str, 2)
Yts_cnn = keras.utils.to_categorical(Yts, 2)

In [6]:
#Form convolutional neural network

model_training = keras.Sequential([
    
    keras.layers.Conv2D(32, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(32, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.15),
    
    keras.layers.Conv2D(64, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.15),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu', use_bias=True),
    keras.layers.Dropout(0.15),

    keras.layers.Dense(2, activation=tf.nn.softmax)
    
])
model_training.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [9]:
model_training.fit(Xtr_cnn,Str_cnn,epochs = 10)

Epoch 1/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.6679 - acc: 0.6014
Epoch 2/10
10000/10000 [==============================] - 9s 906us/step - loss: 0.6480 - acc: 0.6080
Epoch 3/10
10000/10000 [==============================] - 9s 909us/step - loss: 0.6401 - acc: 0.6189
Epoch 4/10
10000/10000 [==============================] - 9s 907us/step - loss: 0.6365 - acc: 0.6293
Epoch 5/10
10000/10000 [==============================] - 9s 908us/step - loss: 0.6332 - acc: 0.6328
Epoch 6/10
10000/10000 [==============================] - 9s 912us/step - loss: 0.6312 - acc: 0.6386
Epoch 7/10
10000/10000 [==============================] - 9s 919us/step - loss: 0.6307 - acc: 0.6393
Epoch 8/10
10000/10000 [==============================] - 9s 915us/step - loss: 0.6257 - acc: 0.6471
Epoch 9/10
10000/10000 [==============================] - 9s 924us/step - loss: 0.6230 - acc: 0.6505
Epoch 10/10
10000/10000 [==============================] - 9s 926us/step - loss: 0.6198 - ac

In [12]:
prediction_train = model_training.predict(Xtr_cnn)
print(prediction_train[:5])
print(prediction_train.shape)

[[0.63380635 0.36619362]
 [0.42604744 0.57395256]
 [0.64660466 0.35339537]
 [0.56657803 0.4334219 ]
 [0.6396351  0.36036494]]
(10000, 2)


In [13]:
p0 = np.min(prediction_train[1]) 
p1 = np.min(prediction_train[0]) 
print(p0)
print(p1)

0.42604744
0.36619362


In [14]:
def generatebeta (Str,prediction,p0,p1): 
    beta = np.zeros((len(prediction ),1)) 
    for i in range(len(prediction)): 
        if Str[i] == 0 : 
            beta[i] = ((prediction[i][0] - p1) / ((1 - p0 - p1) * prediction[i][0]))
        else:
            beta[i] = ((prediction[i][1] - p0) / ((1 - p0 - p1) * prediction[i][1]))
    return beta 

In [15]:
beta = generatebeta (Str,prediction_train,p0,p1)

for i in range(len(beta)):
    if beta[i] < 0:
        beta[i] = 0.0

print(beta[:5]) 

[[2.03231241]
 [0.67619852]
 [0.        ]
 [0.08189549]
 [2.05765417]]


In [19]:
#Form convolutional neural network for important reweighting

model_beta = keras.Sequential([
    
    keras.layers.Conv2D(32, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(32, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.15),
    
    keras.layers.Conv2D(64, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.15),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu', use_bias=True),
    keras.layers.Dropout(0.15),

    keras.layers.Dense(2, activation=tf.nn.softmax)
    
])
model_beta.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [20]:
model_beta.fit(Xtr_cnn,Str_cnn,sample_weight = beta.flatten(),epochs = 10)

Epoch 1/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.3783 - acc: 0.6262
Epoch 2/10
10000/10000 [==============================] - 10s 997us/step - loss: 0.2832 - acc: 0.6499
Epoch 3/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.2594 - acc: 0.6630
Epoch 4/10
10000/10000 [==============================] - 10s 981us/step - loss: 0.2523 - acc: 0.6701
Epoch 5/10
10000/10000 [==============================] - 9s 947us/step - loss: 0.2385 - acc: 0.6748
Epoch 6/10
10000/10000 [==============================] - 10s 952us/step - loss: 0.2334 - acc: 0.6801
Epoch 7/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.2225 - acc: 0.6893
Epoch 8/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.2105 - acc: 0.7016
Epoch 9/10
10000/10000 [==============================] - 10s 1ms/step - loss: 0.2023 - acc: 0.7097
Epoch 10/10
10000/10000 [==============================] - 9s 939us/step - loss: 0.1892 - acc

In [26]:
#Use CNN to predict 
prediction_test_b = model_beta.predict(Xts_cnn)

print(prediction_test_b[:5])
print(prediction_test_b.shape)

pred_Y_b = []

for i in range(len(prediction_test_b)):
    pred_Y_b.append(np.argmax(prediction_test_b[i]))

print(pred_Y_b[:5])

[[9.9904114e-01 9.5881277e-04]
 [9.9510926e-01 4.8907311e-03]
 [9.3825646e-02 9.0617430e-01]
 [9.8678023e-01 1.3219710e-02]
 [7.5820833e-02 9.2417920e-01]]
(2000, 2)
[0, 0, 1, 0, 1]


In [24]:
def computeAccuracy(Y,pred_Y):
    acc = 0.0
    for i in range(len(Y)):
        if Y[i] == pred_Y[i]:
            acc += 1.0
    return acc/len(Y)

In [27]:
acc_b = computeAccuracy(Yts,pred_Y_b)
print(acc_b)

0.8


In [28]:
def generatealpha(p0,p1):
    alpha = ((1-p1+p0) / 2)
    return alpha

In [29]:
alpha = generatealpha(p0,p1)
print(alpha)

0.5299269109964371


In [35]:
alpha_weight = np.zeros((len(Str),1)) 

for i in range(len(Str)):
    if Str[i] == 1:
        alpha_weight[i] = 1-alpha
    else:
        alpha_weight[i] = alpha

print(type(alpha_weight))
print(alpha_weight[:5])

<class 'numpy.ndarray'>
[[0.52992691]
 [0.52992691]
 [0.47007309]
 [0.47007309]
 [0.52992691]]


In [36]:
#Form convolutional neural network for unbaised estimator

model_alpha = keras.Sequential([
    
    keras.layers.Conv2D(32, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(32, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.15),
    
    keras.layers.Conv2D(64, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, (5, 5), use_bias=True, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.15),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu', use_bias=True),
    keras.layers.Dropout(0.15),

    keras.layers.Dense(2, activation=tf.nn.softmax)
    
])
model_alpha.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [37]:
model_alpha.fit(Xtr_cnn,Str_cnn,sample_weight = alpha_weight.flatten(),epochs = 10)

Epoch 1/10
10000/10000 [==============================] - 11s 1ms/step - loss: 0.6507 - acc: 0.6001
Epoch 2/10
10000/10000 [==============================] - 9s 912us/step - loss: 0.6341 - acc: 0.6076
Epoch 3/10
10000/10000 [==============================] - 9s 914us/step - loss: 0.6244 - acc: 0.6227
Epoch 4/10
10000/10000 [==============================] - 9s 920us/step - loss: 0.6269 - acc: 0.6214
Epoch 5/10
10000/10000 [==============================] - 9s 918us/step - loss: 0.6197 - acc: 0.6358
Epoch 6/10
10000/10000 [==============================] - 9s 924us/step - loss: 0.6149 - acc: 0.6463
Epoch 7/10
10000/10000 [==============================] - 9s 933us/step - loss: 0.6093 - acc: 0.6531
Epoch 8/10
10000/10000 [==============================] - 9s 932us/step - loss: 0.6091 - acc: 0.6542
Epoch 9/10
10000/10000 [==============================] - 9s 945us/step - loss: 0.6033 - acc: 0.6617
Epoch 10/10
10000/10000 [==============================] - 9s 936us/step - loss: 0.6010 - ac

In [41]:
prediction_test_a = model_alpha.predict(Xts_cnn)

print(prediction_test_a[:5])
print(prediction_test_a.shape)

pred_Y_a = []

for i in range(len(prediction_test_a)):
    pred_Y_a.append(np.argmax(prediction_test_a[i]))

print(pred_Y_a[:5])

[[0.75482833 0.24517162]
 [0.5299941  0.47000593]
 [0.43163976 0.56836027]
 [0.44965598 0.550344  ]
 [0.49961835 0.5003816 ]]
(2000, 2)
[0, 0, 1, 1, 1]


In [42]:
acc_a = computeAccuracy(Yts,pred_Y_a)
print(acc_a)

0.761
